# Traversability Simulation
## Stucture

```
.
├── agent
│   ├── Agent.py
│   ├── callbacks
│   │   ├── AgentCallback.py
│   │   └── RosBagSaver.py
│   ├── __init__.py
│   └── RospyAgent.py
├── data
├── docker
├── docker-compose.yml
├── env
│   ├── conditions
│   │   ├── __init__.py
│   │   └── __pycache__
│   ├── __init__.py
    ├── krock
│   │   ├── Krock.py
│   │   ├── KrockWebotsEnv.py
│   ├── __pycache__
│   │   └── __init__.cpython-36.pyc
│   └── webots
│       ├── utils.py
│       ├── WebotsEnv.py
│       └── WebotsWorld.py
├── example.ipynb
├── gym.ipynb
├── __init__.py
├── log
├── main.py
├── maps
├── notebooks
│   ├── test.bag
│   └── Visualisation.ipynb
├── parser.py
├── protocols
├── README.md
├── requirements.txt
├── start.sh
└── utils
    ├── History.py
    └── webots2ros
        ├── Supervisor.py

```

A notebook version of this file can be found in `example.ipynb`

## Webots + Krock
![alt text](https://raw.githubusercontent.com/FrancescoSaverioZuppichini/Master-Thesis/gym/resources/images/krock.jpg")

In order to run the Krock robot using Webots we provide a [gym](https://gym.openai.com/) env. If you know gym, then you are good to go. It follows a **basic example** (API could change)

In [4]:
import rospy
import pprint

from env.webots.krock import Krock, KrockWebotsEnv

WORLD_PATH = '/home/francesco/Documents/Master-Thesis/core/env/webots/krock/krock2_ros/worlds/bars1.wbt'
N_STEPS = 4

rospy.init_node("traversability_simulation")
# create our env
env = KrockWebotsEnv.from_file(WORLD_PATH)
# spawn the robot at a random location
init_obs = env.reset()

print('Initial observations:')
pprint.pprint(init_obs)

for _ in range(N_STEPS):
#   go forward for a while
    obs, r, done, _ = env.step(env.GO_FORWARD)
    pprint.pprint(obs)
    if done: break
        
env.step(env.STOP)

/home/francesco/Documents/Master-Thesis/core/env/webots/krock/krock2_ros/worlds/bars1.wbt
Initial observations:
{'sensors': {'front_cam': None,
             'orientation': {'w': -0.6042967846662058,
                             'x': -0.019396150177004034,
                             'y': 0.9990429938527383,
                             'z': -0.039203134977556564},
             'position': {'x': 2.478409671128138,
                          'y': 1.8115926214261309,
                          'z': 0.49422176509154214}}}


({'sensors': {'position': {'x': 2.4789003861146,
    'y': 1.811126550074068,
    'z': 0.43121029420712076},
   'orientation': {'x': -0.04028196616147307,
    'y': 0.9967375122064915,
    'z': -0.06994065314665203,
    'w': -0.5193889768347444},
   'front_cam': None}},
 0,
 False,
 {})

`KrockWebotsEnv` implements `WebotsEnv` that is a generic env for Webots. 

Due to gym interface we can explore the action space and the observation space easily

In [2]:
print(env.action_space)
print(env.observation_space)

Dict(frontal_freq:Box(), lateral_freq:Box())
Dict(sensors:Dict(front_cam:Tuple(Box(600, 800, 3), Box(600, 800, 3)), orientation:Dict(w:Box(1,), x:Box(1,), y:Box(1,), z:Box(1,)), position:Dict(x:Box(1,), y:Box(1,), z:Box(1,))))


Perfect a random action is even esier

In [3]:
obs, r, done, _ = env.step(env.action_space.sample())
env.step(env.STOP)

({'sensors': {'position': {'x': -1.9234729057365247,
    'y': 4.912669606466581,
    'z': 0.1105117153319196},
   'orientation': {'x': 0.00740726304258441,
    'y': 0.9999563198444479,
    'z': 0.005700075207048437,
    'w': -2.071732998847759},
   'front_cam': None}},
 0,
 False,
 {})

If the camera throttle node is activated, `rosrun topic_tools throttle messages /krock/front_camera/image 30` calling `env.render()` will popup a window with the images from the krock's camera

## Agent

The `Agent` class describes something that interacts with the enviroment. It provides a nice interface to adopt.

### Usage

In [4]:
from agent import Agent

class DummyAgent(Agent):
    def act(self, env, *args, **kwargs):
        print('I am moving!')
    
a = DummyAgent()
a()

#### Lifecycle

The agent lifecyle is the following

In [5]:
a() # init
a.act(env) # do something
a.die(env) # RIP, clean up (e.g store stuff empty the state)

I am moving!


You need to first create the agent class and then call it to correctly initialise everything. You can think of it like a *lazy* loader.

### ROS
A `Rospy` agent is already implemented and can be found at `agent.RospyAgent`. It adds two methods to the basic `Agent`: `init_publishers` and `init_subscribers`. They must be overrided to initialise the subscribers and listeners


In [6]:
from agent import RospyAgent
from geometry_msgs.msg import PoseStamped
from sensor_msgs.msg import Joy

class MyRospyAgent(RospyAgent):
    def init_publishers(self):
        return {
            'joy': rospy.Publisher('/joy', Joy, queue_size=1),
        }

    def init_subscribers(self):
        return { 'pose': 
                rospy.Subscriber('/pose', PoseStamped, self.callback_pose)
               }

    def callback_pose(self, data):
        # this will triggers the `on_state_change` method in all callbacks
        self.state['pose'] = data

a = MyRospyAgent()
a()
print(a.subscribers)
print(a.publishers)

{'pose': <rospy.topics.Subscriber object at 0x7f514c7fcc50>}
{'joy': <rospy.topics.Publisher object at 0x7f514c7fcef0>}


### Callbacks
`Agent` implements the protocol `Callbackable` protocol. A callback can be
added by calling the `add_callback` or `add_callbacks`. 

The following interface expose the callback methods for `Agent`:

```python
class AgentCallback():
    def on_state_change(self, agent, key, value):
        pass

    def on_shut_down(self):
        pass
```

In [7]:
from agent.callbacks import AgentCallback

class MyCallback(AgentCallback):
    def on_state_change(self, agent, key, value):
        print(key, value)

Each `Agent` has a `state` that is just a `dict` . Every time it is changed it triggers the `on_state_change` event. This is usefull
to record stuff.

In [8]:
a = DummyAgent()
a.add_callback(MyCallback())
a()

a.state['foo'] = 'foo'

foo foo


#### RosBagSaver

A callback to store the topics of an agent into bags file is implemented

In [9]:
from agent.callbacks import RosBagSaver

a = MyRospyAgent()
a.add_callbacks([RosBagSaver(save_dir='/temp/data', topics=['pose'])])
a()

The files will be stored everytime `agent.die()` il called. A new thread is always spawned to avoid blocking